In [19]:
import os
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt

In [4]:
# 路径
path = input("请输入当前路径：")
os.chdir(path)
print("当前路径为{}".format(os.getcwd()))

当前路径为/Users/wangshuaibo/Documents/北京悦动/脑卒中项目/阿帅笔记/测试数据及分析结果/20230413尊悦IMU传感器与 Xsens MVN数据对比


In [7]:
file = "20230413跺脚走 6 步路测试-zyIMU.csv"
df = pd.read_csv(file)
df.head()

,SensorId,TimeStamp (s),FrameNumber,AccX (g),AccY (g),AccZ (g),GyroX (deg/s),GyroY (deg/s),GyroZ (deg/s),MagX (uT),...,QuatX,QuatY,QuatZ,LinAccX (g),LinAccY (g),LinAccZ (g),Pressure (kPa),Altitude (m),Temperature (degC),HeaveMotion (m)
0,1,-0.0002,0,-0.142809,-0.495929,-0.852164,0.050816,0.057225,-0.027230,14.352529,...,0.249328,-0.114206,-0.185008,0.000048,-0.002238,-0.003064,0.0,0.0,0.0,0.0
1,2,0.0002,0,0.011392,-0.770265,-0.637309,-0.070562,-0.023664,0.105633,43.203743,...,0.111493,0.320466,0.851261,0.000243,-0.000497,0.000171,0.0,0.0,0.0,0.0
2,1,0.0098,1,-0.141649,-0.496640,-0.852689,0.068635,0.039756,-0.117567,14.352529,...,0.249372,-0.114157,-0.185016,-0.001086,-0.001544,-0.002569,0.0,0.0,0.0,0.0
3,2,0.0102,1,0.011884,-0.770451,-0.636944,0.106428,-0.093535,-0.020026,43.876060,...,0.111502,0.320472,0.851256,-0.000237,-0.000191,-0.000232,0.0,0.0,0.0,0.0
4,1,0.0198,2,-0.141504,-0.495902,-0.853171,0.046173,-0.001451,-0.013984,14.352529,...,0.249376,-0.114158,-0.185017,-0.001158,-0.002318,-0.002063,0.0,0.0,0.0,0.0


In [8]:
df.columns

Index(['SensorId', ' TimeStamp (s)', ' FrameNumber', ' AccX (g)', ' AccY (g)',
       ' AccZ (g)', ' GyroX (deg/s)', ' GyroY (deg/s)', ' GyroZ (deg/s)',
       ' MagX (uT)', ' MagY (uT)', ' MagZ (uT)', ' EulerX (deg)',
       ' EulerY (deg)', ' EulerZ (deg)', ' QuatW', ' QuatX', ' QuatY',
       ' QuatZ', ' LinAccX (g)', ' LinAccY (g)', ' LinAccZ (g)',
       ' Pressure (kPa)', ' Altitude (m)', ' Temperature (degC)',
       ' HeaveMotion (m)'],
      dtype='object')

In [11]:
Quats = df.loc[:,[" QuatW"," QuatX"," QuatY"," QuatZ"]]
Eulers = df.loc[:,[' EulerX (deg)',' EulerY (deg)', ' EulerZ (deg)']]
Quats

,QuatW,QuatX,QuatY,QuatZ
0,-0.943697,0.249328,-0.114206,-0.185008
1,-0.400282,0.111493,0.320466,0.851261
2,-0.943690,0.249372,-0.114157,-0.185016
3,-0.400286,0.111502,0.320472,0.851256
4,-0.943689,0.249376,-0.114158,-0.185017
...,...,...,...,...
3920,-0.959692,0.266356,-0.076594,-0.046684
3921,-0.959691,0.266359,-0.076586,-0.046694
3922,-0.959690,0.266366,-0.076572,-0.046705
3923,-0.959689,0.266367,-0.076572,-0.046718


In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation

def transform_euler_angles(quaternion):
    """
    使用四元数转换欧拉角，并将其转换为特定形式的欧拉角。
    
    Args:
        quaternion (list): 长度为4的列表，表示四元数。
    
    Returns:
        list: 长度为3的列表，表示经过转换的欧拉角。
    """
    # 使用四元数转换欧拉角
    rotation = Rotation.from_quat(quaternion)
    euler_angles = rotation.as_euler("zyx", degrees=True)
    
    # 将欧拉角转换为特定形式
    # 仅使用 transform 转化，得到的数据不正确，需要进一步转化
    # 第一个角度不需要变化，第二个角度前面加负数，第三个是和180互补加负数
    transformed_angles = np.round([euler_angles[0], -euler_angles[1], -(180 - euler_angles[2])], 6)
    
    return transformed_angles

In [79]:
# 此段代码可以不执行，意在演示 df.iterrows 方法弊端
# 使用 df.iterrows() 对每列的数据进行转换
"""
使用 df.iterrows() 存在的问题
# 1. 当数据集比较大，转换比较慢，因为需要在 python 和 Pandas 之间转换
# 2. 需要构建空的 DataFrame 接收新生成的数据集

相比之下 使用 df.appply() 进行转换速度就要快很多
apply() 方法允许在 Pandas 数据框架的每行或者每列上应用函数
它在 Pandas 中使用 Cython 完成，因而速度更快

"""
for index, row in Quats.iterrows():
    q = row[[" QuatW"," QuatX"," QuatY"," QuatZ"]]
    euler_tran = transform_euler_angles(q)
print(euler_tran)


[31.488617 -7.012192 -7.55418 ]


In [100]:
# 使用 df.apply() 方式进行转换
Eulers_trans = Quats.apply(transform_euler_angles,axis=1)
# 对转换后得到的数据集进行命名
# 先讲列表转换为元组再进行命名
Eulers_trans.name = tuple([' EulerX (deg)',' EulerY (deg)', ' EulerZ (deg)'])
Eulers_trans

"""
下面的包含 df.apply() 以及匿名函数 lamdba() 的使用
df.apply() 可以将同一行的操作复用到每一行
因而我们的主要目的是如何从一行数据当中提取出对应的数据

查看第一行的数据
Eulers_trans.head(1)
0    [31.116637, -7.082408, -24.158045]
Name: ( EulerX (deg),  EulerY (deg),  EulerZ (deg)), dtype: object

我们发现数据放在一个列表中，我们需要做的是依次把列表中的数据提取出来
此时构建 lamda 函数
lambda x:x[0] 即可分别将函数提取出来
lambda 函数返回的为为函数本身
"""
Euler_trans_df = pd.DataFrame({
    "Euler X(degress)":[Eulers_trans.apply(lambda x:x[0])],
    "Euler Y(degress)":[Eulers_trans.apply(lambda x:x[1])],
    "Euler Z(degress)":[Eulers_trans.apply(lambda x:x[2])]
})

Eulers_trans_df

,EulerX (deg),EulerY (deg),EulerZ (deg)
0,31.116637,-7.082408,-24.158045
1,39.514719,3.826480,-228.993119
2,31.120353,-7.075796,-24.157555
3,39.515761,3.825984,-228.993896
4,31.120830,-7.075744,-24.157677
...,...,...,...
3920,31.486789,-7.015903,-7.549953
3921,31.487053,-7.014673,-7.550839
3922,31.487849,-7.012690,-7.551664
3923,31.487995,-7.012340,-7.553077


In [109]:
# lambda 函数演示
x = [31.116637, -7.082408, -24.158045]
def x_list(x):
    return x[0]
x_list(x)

# 上述内容可以简化为
lambda x: x[0]

secs = pd.DataFrame({
    "时间(秒）": [60,120,180]
})

# 将时间的秒转换为分钟
mins = secs.apply(lambda x: x/60)
mins

# 如果不用 lambda 函数
def sec2min(sec):
    return sec/60

mins = secs.apply(sec2min)
mins

# 函数太短不值当定义, lambda 合适

,时间(秒）
0,1.0
1,2.0
2,3.0
